Integrantes:
Lucas Pereira Carvalho
Lucas 

In [48]:
import pyspark
from pyspark.sql import SparkSession
from delta.tables import DeltaTable

spark = SparkSession.builder \
    .appName("spotify-datalake") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.executor.instances", "2") \
    .config("spark.executor.cores", "2") \
    .config("spark.executor.memory", "1024M") \
    .config("spark.ui.port", "4061") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")
sc = spark.sparkContext

In [49]:
playlists_v1_path = '/shared/sampled/playlists_v1.json'
playlists_v1_df = spark.read.json(playlists_v1_path)
tracks_v1_path = '/shared/sampled/tracks_v1.json'
tracks_v1_df = spark.read.json(tracks_v1_path)

# Task 1A:
## Silver layer:
  
Foram criadas as seguintes tabelas para a camada silver e seus respectivos esquemas:

- song_info
    name (string): Nome da música
    track_uri (string): URI da música
    duration_ms (long): Duração da música em ms
    album_uri (string): URI do album
    artist_uri (string): URI do artista

- album_info
    name (string): Nome do album
    album_uri (string): URI do album
    artist_uri (string): URI do artista

- artist_info
    name (string): Nome do artista
    artist_uri (string): URI do artista

- playlist_info
    name (string): Nome da playlist
    playlist_id (int): ID da playlist
    description (string): Descrição da playlist
    is_collaborative (boolean): Indica se a playlist é colaborativa

- playlist_tracks
    playlist_id (int): ID da playlist
    position (int): Posição da música na playlist
    track_uri (string): URI da música
    artist_uri (string): URI do artista
    album_uri (string): URI do álbum

Nesta etapa, foram realizadas as seguintes modificações:

- Seleção de colunas relevantes:
  Extraída apenas as colunas necessárias para cada tabela na camada Silver, removendo colunas desnecessárias
- Remoção de duplicatas:
  Aplicado distinct() nas tabelas para evitar duplicatas
- Renomeação de colunas:
  Renomeada colunas para nomes mais descritivos, como track_name para name
- Conversão de tipos de dados:
  Convertido colunas como collaborative para boolean, facilitando a computação em cima dos dados

In [50]:
from pyspark.sql import functions as F

song_info_df = tracks_v1_df.select(
    F.col("track_name").alias("name"),
    F.col("track_uri").alias("track_uri"),
    F.col("duration_ms").alias("duration_ms"),
    F.col("album_uri").alias("album_uri"),
    F.col("artist_uri").alias("artist_uri")
)

song_info_df.write.format("parquet").mode("overwrite").save("datalake/silver/song_info")

25/02/07 22:51:26 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
                                                                                

In [51]:
album_info_df = tracks_v1_df.select(
    F.col("album_name").alias("name"),
    F.col("album_uri").alias("album_uri"),
    F.col("artist_uri").alias("artist_uri")
).distinct()

album_info_df.write.format("parquet").mode("overwrite").save("datalake/silver/album_info")

In [52]:
artist_info_df = tracks_v1_df.select(
    F.col("artist_name").alias("name"),
    F.col("artist_uri").alias("artist_uri")
).distinct()

artist_info_df.write.format("parquet").mode("overwrite").save("datalake/silver/artist_info")

In [53]:
playlist_info_df = playlists_v1_df.select(
    F.col("name").alias("name"),
    F.col("pid").alias("pid"),
    F.col("description").alias("description"),
    F.col("collaborative").alias("is_collaborative")
)

playlist_info_df.write.format("parquet").mode("overwrite").save("datalake/silver/playlist_info")

In [54]:
playlist_tracks_df = tracks_v1_df.select(
    F.col("pid").alias("playlist_id"),
    F.col("pos").alias("position"),
    F.col("track_uri").alias("track_uri"),
    F.col("artist_uri").alias("artist_uri"),
    F.col("album_uri").alias("album_uri")
)

playlist_tracks_df.write.format("parquet").mode("overwrite").save("datalake/silver/playlist_tracks")

25/02/07 22:51:29 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


## Gold layer:

Foram criadas as seguintes tabelas para a camada gold e seus respectivos esquemas:

- playlist_info_gold:
    name (string): Nome da playlist
    playlist_id (int): ID da playlist
    description (string): Descrição da playlist
    total_duration_ms (long): Duração total da playlist em ms
    num_tracks (int): Numero de msuicas na playlist
    num_artists (int): Numero de artistas únicos na playlist
    num_albums (int): Numero de álbuns únicos na playlist

- playlist_tracks_gold
    playlist_id (int): ID da playlist
    position (int): Posição da música na playlist
    song_name (string): Nome da música
    album_name (string): Nome do álbum
    artist_name (string): Nome do artista

Nesta etapa foram feitas as seguintes transformações nos dados:
- Calculada métricas total_duration_ms, num_tracks, num_artists, e num_albums de forma agregada para a tabela playlist_info_gold
- Combinada as tabelas playlist_tracks, song_info, album_info, e artist_info para criar playlist_tracks_gold
- Ordenado as músicas em playlist_tracks_gold pela coluna position

In [55]:
playlist_info_gold_df = tracks_v1_df.groupBy("pid").agg(
    F.count("track_uri").alias("num_tracks"),
    F.sum("duration_ms").alias("total_duration_ms"),
    F.countDistinct("artist_uri").alias("num_artists"),
    F.countDistinct("album_uri").alias("num_albums")
).join(playlist_info_df, "pid", "inner")

playlist_info_gold_df.write.format("parquet").mode("overwrite").save("datalake/gold/playlist_info")

In [56]:
song_info_df = song_info_df.withColumnRenamed("name", "song_name")
album_info_df = album_info_df.withColumnRenamed("name", "album_name")
artist_info_df = artist_info_df.withColumnRenamed("name", "artist_name")

playlist_tracks_gold_df = playlist_tracks_df.join(
    song_info_df, "track_uri", "inner"
).join(
    album_info_df, "album_uri", "inner"
).join(
    artist_info_df, "artist_uri", "inner"
).select(
    "playlist_id",
    "position",
    "song_name",
    "album_name",
    "artist_name"
)

# Task 1B:

Nesta tarefa, foram salvas as tabelas playlist_info_gold_df e playlist_tracks_gold_df em ambos os formatos, json e parquet.
Após isso, com auxílio da lib time, contamos o tempo levado para fazer a leitura do dataframe, para ambos os formatos. Pôde-se verificar um melhor desempenho no formato parquet, conforme esperado, já que se trata de um formato mais otimizado para armazenamento e leitura dos dados, em comparação ao json.
Código usado para possível visualização dessa diferença de tempo a seguir:

In [57]:
playlist_info_gold_df.write.format("json").mode("overwrite").save("datalake/gold_json/playlist_info")
playlist_tracks_gold_df.write.format("json").mode("overwrite").save("datalake/gold_json/playlist_tracks")

In [58]:
playlist_info_gold_df.write.format("parquet").mode("overwrite").save("datalake/gold_parquet/playlist_info")
playlist_tracks_gold_df.write.format("parquet").mode("overwrite").save("datalake/gold_parquet/playlist_tracks")

25/02/07 22:51:54 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/02/07 22:52:01 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
                                                                                

In [59]:
import time

# tempo de load em json
start_time = time.time()
json_df = spark.read.json("datalake/gold_json/playlist_info")
json_df.count()
json_load_time = time.time() - start_time

print(f"Tempo de carregamento (JSON): {json_load_time} segundos")

# tempo de load em parquet
start_time = time.time()
parquet_df = spark.read.parquet("datalake/gold_parquet/playlist_info")
parquet_df.count()
parquet_load_time = time.time() - start_time

print(f"Tempo de carregamento (Parquet): {parquet_load_time} segundos")

Tempo de carregamento (JSON): 0.25846004486083984 segundos
Tempo de carregamento (Parquet): 0.16401124000549316 segundos


**Task 2**

In [75]:
bronze_path = "datalake/bronze/"
silver_path = "datalake/silver/"
gold_path = "datalake/gold_parquet/"

def get_song_info(tracks_v):
    return tracks_v.select(
        F.col("track_name").alias("name"),
        F.col("track_uri").alias("track_uri"),
        F.col("duration_ms").alias("duration_ms"),
        F.col("album_uri").alias("album_uri"),
        F.col("artist_uri").alias("artist_uri")
    ).distinct()

def get_album_info(tracks_v):
    return tracks_v.select(
        F.col("album_name").alias("name"),
        F.col("album_uri").alias("album_uri"),
        F.col("artist_uri").alias("artist_uri")
    ).distinct()

def get_artist_info(tracks_v):
    return tracks_v.select(
        F.col("artist_name").alias("name"),
        F.col("artist_uri").alias("artist_uri")
    ).distinct()

def get_playlist_info(playlists_v):
    return playlists_v.select(
        F.col("name").alias("name"),
        F.col("pid").alias("pid"),
        F.col("description").alias("description"),
        F.col("collaborative").alias("is_collaborative")
    ).distinct()

def get_playlist_track_info(tracks_v):
    return tracks_v.select(
        F.col("pid").alias("playlist_id"),
        F.col("pos").alias("position"),
        F.col("track_uri").alias("track_uri"),
        F.col("artist_uri").alias("artist_uri"),
        F.col("album_uri").alias("album_uri")
    )

def ingest_new_data(version):
    playlists = spark.read.json(f"/shared/sampled/playlists_{version}.json")
    tracks = spark.read.json(f"/shared/sampled/tracks_{version}.json")

    playlists.write.mode("append").parquet(bronze_path + "playlists/")
    tracks.write.mode("append").parquet(bronze_path + "tracks/")

    return playlists, tracks

def update_silver_layer(playlists_v, tracks_v):
    
    song_info = spark.read.parquet(silver_path + "song_info")
    album_info = spark.read.parquet(silver_path + "album_info")
    artist_info = spark.read.parquet(silver_path + "artist_info")
    playlist_info = spark.read.parquet(silver_path + "playlist_info")
    playlist_tracks = spark.read.parquet(silver_path + "playlist_tracks")

    new_song_info = get_song_info(tracks_v)
    song_info = song_info.unionByName(new_song_info, allowMissingColumns=True).dropDuplicates(["track_uri"])

    new_album_info = get_album_info(tracks_v)
    album_info = album_info.unionByName(new_album_info, allowMissingColumns=True).dropDuplicates(["album_uri"])

    new_artist_info = get_artist_info(tracks_v)
    artist_info = artist_info.unionByName(new_artist_info, allowMissingColumns=True).dropDuplicates(["artist_uri"])

    new_playlist_info = get_playlist_info(playlists_v)
    playlist_info = playlist_info.unionByName(new_playlist_info, allowMissingColumns=True).dropDuplicates(["pid"])

    new_playlist_tracks = get_playlist_track_info(tracks_v)
    playlist_tracks = playlist_tracks.unionByName(new_playlist_tracks, allowMissingColumns=True).dropDuplicates(["playlist_id", "track_uri"])

    playlist_info = playlist_info.withColumn(
        "name", F.when(F.col("pid") == "11992", "GYM WORKOUT").otherwise(F.col("name"))
    ).withColumn(
        "is_collaborative", F.when(F.col("pid") == "11992", F.lit(True).cast("string")).otherwise(F.col("is_collaborative"))
    )

    song_info.write.mode("overwrite").parquet(silver_path + "song_info")
    album_info.write.mode("overwrite").parquet(silver_path + "album_info")
    artist_info.write.mode("overwrite").parquet(silver_path + "artist_info")
    playlist_info.write.mode("overwrite").parquet(silver_path + "playlist_info")
    playlist_tracks.write.mode("overwrite").parquet(silver_path + "playlist_tracks")

def update_gold_layer():
    playlist_info = spark.read.parquet(silver_path + "playlist_info")
    song_info = spark.read.parquet(silver_path + "song_info")
    album_info = spark.read.parquet(silver_path + "album_info")
    artist_info = spark.read.parquet(silver_path + "artist_info")
    playlist_tracks = spark.read.parquet(silver_path + "playlist_tracks")

    song_info = song_info_df.withColumnRenamed("name", "song_name")
    album_info = album_info_df.withColumnRenamed("name", "album_name")
    artist_info = artist_info_df.withColumnRenamed("name", "artist_name")

    playlists_gold = playlist_info.alias("pi") \
        .join(playlist_tracks.alias("pt"), F.col("pi.pid") == F.col("pt.playlist_id"), "left") \
        .join(song_info.alias("si"), F.col("pt.track_uri") == F.col("si.track_uri"), "left") \
        .groupBy("pi.pid", "pi.name", "pi.description") \
        .agg(
            F.count("pt.track_uri").alias("num_tracks"),
            F.sum("si.duration_ms").alias("total_duration_ms"),
            F.countDistinct("pt.artist_uri").alias("num_artists"),
            F.countDistinct("pt.album_uri").alias("num_albums")
    )

    playlist_tracks_gold = playlist_tracks.join(
            song_info.alias("si"), "track_uri", "inner"
        ).join(
            album_info.alias("ali"), "album_uri", "inner"
        ).join(
            artist_info.alias("ari"), "artist_uri", "inner"
        ).select(
            "playlist_id",
            "position",
            "song_name",
            "album_name",
            "artist_name"
    )

    playlists_gold.write.mode("overwrite").parquet(gold_path + "playlist_info")
    playlist_tracks_gold.write.mode("overwrite").parquet(gold_path + "playlist_tracks")


In [76]:
def run_pipeline(version_path):
    print(f"Iniciando ingestão dos dados de {version_path}...")
    playlists, tracks = ingest_new_data(version_path)

    print("Atualizando a camada Silver...")
    update_silver_layer(playlists, tracks)

    print("Atualizando a camada Gold...")
    update_gold_layer()

    print(f"Pipeline concluído para {version_path}!")

In [77]:
run_pipeline("v2")

Iniciando ingestão dos dados de v2...


25/02/07 22:57:25 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
                                                                                

Atualizando a camada Silver...


25/02/07 22:57:30 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/02/07 22:57:31 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/02/07 22:57:35 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
                                                                                

Atualizando a camada Gold...


25/02/07 22:57:40 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/07 22:57:40 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/07 22:57:40 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/07 22:57:40 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/07 22:57:40 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/07 22:57:40 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/07 22:57:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/07 22:57:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/07 22:57:41 WARN RowBasedKeyValueBatch: Calling spill() on

Pipeline concluído para v2!


In [78]:
run_pipeline("v3")

Iniciando ingestão dos dados de v3...


25/02/07 22:58:17 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


Atualizando a camada Silver...


25/02/07 22:58:20 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/02/07 22:58:21 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/02/07 22:58:24 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
                                                                                

Atualizando a camada Gold...


25/02/07 22:58:28 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/07 22:58:28 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/07 22:58:28 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/07 22:58:28 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/07 22:58:28 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/07 22:58:29 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/07 22:58:29 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/07 22:58:29 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/07 22:58:31 WARN RowBasedKeyValueBatch: Calling spill() on

Pipeline concluído para v3!


Nessa etapa, o principal desafio no uso do formato Parquet para manipulação dos dados foi a tentativa de reduzir ao máximo o overhead de escrever todos os arquivos novamente. Isso acarreta em um tempo de execução e consumo de recursos maiores. Com isso, por não suportar operações de update, a alternativa encontrada foi realizar um **unionByName** entre os dados antigos e os novos removendo duplicações. Com isso esperamos que dados que não foram modificados não precisem ser processados.

**Task 3**

In [89]:
bronze_path = "datalake/bronze/"
silver_path = "datalake/silver_delta/"
gold_path = "datalake/gold_delta/"

def delta_table_exists(path):
    return os.path.exists(path) and bool(os.listdir(path))

def initialize_delta_table(df, path):
    df.write.format("delta").mode("overwrite").save(path)

def get_song_info(tracks_v):
    return tracks_v.select(
        F.col("track_name").alias("name"),
        F.col("track_uri").alias("track_uri"),
        F.col("duration_ms").alias("duration_ms"),
        F.col("album_uri").alias("album_uri"),
        F.col("artist_uri").alias("artist_uri")
    ).distinct()

def get_album_info(tracks_v):
    return tracks_v.select(
        F.col("album_name").alias("name"),
        F.col("album_uri").alias("album_uri"),
        F.col("artist_uri").alias("artist_uri")
    ).distinct()

def get_artist_info(tracks_v):
    return tracks_v.select(
        F.col("artist_name").alias("name"),
        F.col("artist_uri").alias("artist_uri")
    ).distinct()

def get_playlist_info(playlists_v):
    return playlists_v.select(
        F.col("name").alias("name"),
        F.col("pid").alias("pid"),
        F.col("description").alias("description"),
        F.col("collaborative").alias("is_collaborative")
    ).distinct()

def get_playlist_track_info(tracks_v):
    return tracks_v.select(
        F.col("pid").alias("playlist_id"),
        F.col("pos").alias("position"),
        F.col("track_uri").alias("track_uri"),
        F.col("artist_uri").alias("artist_uri"),
        F.col("album_uri").alias("album_uri")
    )

def update_gym_workout(playlist_info):
    playlist_info = playlist_info.withColumn(
        "name", F.when(F.col("pid") == "11992", "GYM WORKOUT").otherwise(F.col("name"))
    ).withColumn(
        "is_collaborative", F.when(F.col("pid") == "11992", F.lit(True).cast("string")).otherwise(F.col("is_collaborative"))
    )

def update_gym_workout_with_delta(playlist_info_delta):
    playlist_info_delta.update(
        condition=F.col("pid") == 11992,
        set={"name": F.lit("GYM WORKOUT"), "is_collaborative": F.lit(True)}
    )
    
def ingest_new_data(version):
    playlists = spark.read.json(f"/shared/sampled/playlists_{version}.json")
    tracks = spark.read.json(f"/shared/sampled/tracks_{version}.json")

    playlists.write.format("delta").mode("append").save(bronze_path + "playlists/")
    tracks.write.format("delta").mode("append").save(bronze_path + "tracks/")

    return playlists, tracks

def update_silver_layer(playlists_v, tracks_v):

    new_song_info = get_song_info(tracks_v)
    new_album_info = get_album_info(tracks_v)
    new_artist_info = get_artist_info(tracks_v)
    new_playlist_info = get_playlist_info(playlists_v)
    new_playlist_tracks = get_playlist_track_info(tracks_v)

    if not delta_table_exists(silver_path):
        update_gym_workout(new_playlist_info)

        initialize_delta_table(new_song_info, silver_path + "song_info")
        initialize_delta_table(new_album_info, silver_path + "album_info")
        initialize_delta_table(new_artist_info, silver_path + "artist_info")
        initialize_delta_table(new_playlist_info, silver_path + "playlist_info")
        initialize_delta_table(new_playlist_tracks, silver_path + "playlist_tracks")
        return

    song_info = DeltaTable.forPath(spark, silver_path + "song_info")
    album_info = DeltaTable.forPath(spark, silver_path + "album_info")
    artist_info = DeltaTable.forPath(spark, silver_path + "artist_info")
    playlist_info = DeltaTable.forPath(spark, silver_path + "playlist_info")
    playlist_tracks = DeltaTable.forPath(spark, silver_path + "playlist_tracks")

    def merge_delta(table, new_data, join_key):
        table.alias("old").merge(
            new_data.alias("new"), f"old.{join_key} = new.{join_key}"
        ).whenNotMatchedInsertAll().execute()

    merge_delta(song_info, new_song_info, "track_uri")
    merge_delta(album_info, new_album_info, "album_uri")
    merge_delta(artist_info, new_artist_info, "artist_uri")
    merge_delta(playlist_info, new_playlist_info, "pid")
    merge_delta(playlist_tracks, new_playlist_tracks, "track_uri")

    update_gym_workout_with_delta(playlist_info)
    
def update_gold_layer():
    playlist_info = spark.read.format("delta").load(silver_path + "playlist_info")
    song_info = spark.read.format("delta").load(silver_path + "song_info")
    album_info = spark.read.format("delta").load(silver_path + "album_info")
    artist_info = spark.read.format("delta").load(silver_path + "artist_info")
    playlist_tracks = spark.read.format("delta").load(silver_path + "playlist_tracks")

    song_info = song_info_df.withColumnRenamed("name", "song_name")
    album_info = album_info_df.withColumnRenamed("name", "album_name")
    artist_info = artist_info_df.withColumnRenamed("name", "artist_name")

    playlists_gold = playlist_info.alias("pi") \
        .join(playlist_tracks.alias("pt"), F.col("pi.pid") == F.col("pt.playlist_id"), "left") \
        .join(song_info.alias("si"), F.col("pt.track_uri") == F.col("si.track_uri"), "left") \
        .groupBy("pi.pid", "pi.name", "pi.description") \
        .agg(
            F.count("pt.track_uri").alias("num_tracks"),
            F.sum("si.duration_ms").alias("total_duration_ms"),
            F.countDistinct("pt.artist_uri").alias("num_artists"),
            F.countDistinct("pt.album_uri").alias("num_albums")
        )

    playlist_tracks_gold = playlist_tracks.join(
            song_info.alias("si"), "track_uri", "inner"
        ).join(
            album_info.alias("ali"), "album_uri", "inner"
        ).join(
            artist_info.alias("ari"), "artist_uri", "inner"
        ).select(
            "playlist_id",
            "position",
            "song_name",
            "album_name",
            "artist_name"
    )

    playlists_gold.write.format("delta").mode("overwrite").save(gold_path + "playlist_info")
    playlist_tracks_gold.write.format("delta").mode("overwrite").save(gold_path + "playlist_tracks")


In [86]:
def run_pipeline(version_path):
    print(f"Iniciando ingestão dos dados de {version_path}...")
    playlists, tracks = ingest_new_data(version_path)

    print("Atualizando a camada Silver...")
    update_silver_layer(playlists, tracks)

    print("Atualizando a camada Gold...")
    update_gold_layer()

    print(f"Pipeline concluído para {version_path}!")

In [87]:
run_pipeline("v2")

Iniciando ingestão dos dados de v2...


25/02/07 23:01:29 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
                                                                                

Atualizando a camada Silver...


25/02/07 23:01:32 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/02/07 23:01:35 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/02/07 23:01:40 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
                                                                                

Atualizando a camada Gold...


25/02/07 23:01:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/07 23:01:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/07 23:01:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/07 23:01:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/07 23:01:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/07 23:01:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/07 23:01:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/07 23:01:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/07 23:01:47 WARN RowBasedKeyValueBatch: Calling spill() on

Pipeline concluído para v2!


In [90]:
run_pipeline("v3")

Iniciando ingestão dos dados de v3...


25/02/07 23:03:13 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
                                                                                

Atualizando a camada Silver...


25/02/07 23:03:15 WARN MergeIntoCommand: Merge source has SQLMetric(id: 53286, name: Some(number of source rows), value: 126481) rows in initial scan but SQLMetric(id: 53287, name: Some(number of source rows (during repeated scan)), value: 0) rows in second scan
25/02/07 23:03:16 WARN MergeIntoCommand: Merge source has SQLMetric(id: 53574, name: Some(number of source rows), value: 70477) rows in initial scan but SQLMetric(id: 53575, name: Some(number of source rows (during repeated scan)), value: 0) rows in second scan
25/02/07 23:03:16 WARN MergeIntoCommand: Merge source has SQLMetric(id: 53862, name: Some(number of source rows), value: 30715) rows in initial scan but SQLMetric(id: 53863, name: Some(number of source rows (during repeated scan)), value: 0) rows in second scan
25/02/07 23:03:17 WARN MergeIntoCommand: Merge source has SQLMetric(id: 54150, name: Some(number of source rows), value: 42014) rows in initial scan but SQLMetric(id: 54151, name: Some(number of source rows (durin

Atualizando a camada Gold...


25/02/07 23:03:23 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/07 23:03:23 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/07 23:03:23 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/07 23:03:23 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/07 23:03:23 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/07 23:03:23 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/07 23:03:23 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/07 23:03:23 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/07 23:03:24 WARN RowBasedKeyValueBatch: Calling spill() on

Pipeline concluído para v3!


In [95]:
import os

def get_storage_size(path):
    total_size = 0
    for root, _, files in os.walk(path):
        for file in files:
            file_path = os.path.join(root, file)
            total_size += os.path.getsize(file_path)
    return total_size / (1024 * 1024)  # Convert bytes to MB

silver_parquet_path = "datalake/gold_parquet/"
silver_delta_path = "datalake/gold_delta/"

parquet_size = get_storage_size(silver_parquet_path)
delta_size = get_storage_size(silver_delta_path)

print(f"Total storage used:")
print(f" - Parquet: {parquet_size:.2f} MB")
print(f" - Delta  : {delta_size:.2f} MB")
print(f"Delta is {(parquet_size - delta_size) / parquet_size * 100:.2f}% smaller than Parquet.")


Total storage used:
 - Parquet: 26.45 MB
 - Delta  : 24.61 MB
Delta is 6.98% smaller than Parquet.


Como podemos observar, o uso do formato Delta contribuiu para melhorar o processo de atualização das camadas. Isso ocorreu com uma diminuição no tempo necessário para atualizá-las e também com uma redução no tamanho total ocupado pelos arquivos gerados em aproximadamente 7% se comparado ao processo utilizando Parquet.